<a href="https://colab.research.google.com/github/NamanGautam0404/NG.pro/blob/main/DiseasePredictionusingNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from collections import Counter as ctr
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/mac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mac/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
df = pd.read_csv('Symptom2Disease.csv')
df.head(10)

,Unnamed: 0,label,text
0,0,Psoriasis,I have been experiencing a skin rash on my arm...
1,1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,2,Psoriasis,I have been experiencing joint pain in my fing...
3,3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,4,Psoriasis,"My nails have small dents or pits in them, and..."
5,5,Psoriasis,The skin on my palms and soles is thickened an...
6,6,Psoriasis,"The skin around my mouth, nose, and eyes is re..."
7,7,Psoriasis,My skin is very sensitive and reacts easily to...
8,8,Psoriasis,I have noticed a sudden peeling of skin at dif...
9,9,Psoriasis,The skin on my genitals is red and inflamed. I...


In [ ]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
ctr(df['label'])

Counter({'Psoriasis': 50,
         'Varicose Veins': 50,
         'Typhoid': 50,
         'Chicken pox': 50,
         'Impetigo': 50,
         'Dengue': 50,
         'Fungal infection': 50,
         'Common Cold': 50,
         'Pneumonia': 50,
         'Dimorphic Hemorrhoids': 50,
         'Arthritis': 50,
         'Acne': 50,
         'Bronchial Asthma': 50,
         'Hypertension': 50,
         'Migraine': 50,
         'Cervical spondylosis': 50,
         'Jaundice': 50,
         'Malaria': 50,
         'urinary tract infection': 50,
         'allergy': 50,
         'gastroesophageal reflux disease': 50,
         'drug reaction': 50,
         'peptic ulcer disease': 50,
         'diabetes': 50})

In [ ]:
df.sample(10)

,label,text
849,Jaundice,"I've been losing weight, feeling really fatigu..."
750,Cervical spondylosis,"I have been experiencing severe back pain, a p..."
1155,diabetes,I have a feeling of tremors and muscle twitchi...
1075,drug reaction,My rashes occasionally cause my skin to peel o...
751,Cervical spondylosis,"Along with a chronic cough and limb weakness, ..."
1006,gastroesophageal reflux disease,I have a loss of appetite and difficulty swall...
967,allergy,I'm always worn out and experiencing a tingle ...
212,Impetigo,"Initially, I developed small red sores near my..."
284,Dengue,I have a high fever along with a severe headac...
443,Pneumonia,I'm running a temperature and can't seem to ca...


In [ ]:
def preprocess_text(text):
    tokens = word_tokenize(text)
    snowball_stemmer = SnowballStemmer('english')
    tokens = [snowball_stemmer.stem(token.lower()) for token in tokens if token.isalpha()]
    return ' '.join(tokens)

df['text'] = df['text'].apply(preprocess_text)

In [ ]:
df.sample(5)

,label,text
915,urinary tract infection,my is pee is dark and often bloodi and often h...
750,Cervical spondylosis,i have been experienc sever back pain a persis...
626,Bronchial Asthma,recent i have had a number of concern symptom ...
281,Dengue,i have been experienc chill and shiver there i...
1064,drug reaction,my skin occasion peel off the rash that i have...


In [ ]:
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(df['text'])

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
base_models = [
    ('nb', MultinomialNB()),
    ('rf', RandomForestClassifier()),
    ('lr', LogisticRegression()),
    ('svm', SVC(kernel='linear', probability=True))
]

In [ ]:
voting_classifier = VotingClassifier(estimators=base_models, voting='hard')

In [ ]:
voting_classifier.fit(X_train, y_train)

VotingClassifier(estimators=[('nb', MultinomialNB()),
                             ('rf', RandomForestClassifier()),
                             ('lr', LogisticRegression()),
                             ('svm', SVC(kernel='linear', probability=True))])

In [ ]:
accuracy = voting_classifier.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.9708333333333333


In [ ]:
import joblib

joblib.dump(voting_classifier, 'voting_classifier_model_Disease_pred_97_percent_acc.pkl')

['voting_classifier_model_Disease_pred_97_percent_acc.pkl']

In [ ]:
loaded_model = joblib.load('voting_classifier_model_Disease_pred_97_percent_acc.pkl')

In [ ]:
# Sample text
sample_text = "I have been experiencing a skin rash on my arm for the past few weeks."
sample_text_processed = preprocess_text(sample_text)
sample_text_transformed = tfidf_vectorizer.transform([sample_text_processed])
predicted_label = label_encoder.inverse_transform(voting_classifier.predict(sample_text_transformed))

print("Predicted Label:", predicted_label)

Predicted Label: ['Psoriasis']


In [ ]:
text = 'i been realli weari and ill i been suffer from..'

sample_text = text
sample_text_processed = preprocess_text(sample_text)
sample_text_transformed = tfidf_vectorizer.transform([sample_text_processed])
predicted_label = label_encoder.inverse_transform(voting_classifier.predict(sample_text_transformed))

print("Predicted Label:", predicted_label)

Predicted Label: ['Common Cold']


In [ ]:
df.sample()

,label,text
685,Hypertension,i have been deal with a headach and chest pain...


In [ ]:
# Compute accuracy
accuracy = accuracy_score(y_test, y_test)
print("Accuracy:", accuracy)

# Compute precision
precision = precision_score(y_test, y_test, average='macro')  # 'macro' computes precision for each label, and returns the average
print("Precision:", precision)

# Compute recall
recall = recall_score(y_test, y_test, average='macro')  # 'macro' computes recall for each label, and returns the average
print("Recall:", recall)

# Compute F1-score
f1 = f1_score(y_test, y_test, average='macro')  # 'macro' computes F1-score for each label, and returns the average
print("F1-score:", f1)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_test)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-score: 1.0
Confusion Matrix:
[[ 7  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 11  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  7  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  7  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 13  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 11  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 11  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  

In [ ]:
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer_disease_nlp.joblib')
joblib.dump(label_encoder, 'label_encoder_disease_nlp.joblib')

['label_encoder_disease_nlp.joblib']

In [ ]:
import joblib
voting_classifier = joblib.load('voting_classifier_model_Disease_pred_97_percent_acc.pkl')
tfidf_vectorizer = joblib.load('tfidf_vectorizer_disease_nlp.joblib')
label_encoder = joblib.load('label_encoder_disease_nlp.joblib')

In [ ]:
import joblib
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

# Load the model and vectorizer
voting_classifier = joblib.load('voting_classifier_model_Disease_pred_97_percent_acc.pkl')
tfidf_vectorizer = joblib.load('tfidf_vectorizer_disease_nlp.joblib')
label_encoder = joblib.load('label_encoder_disease_nlp.joblib')

# Sample text
sample_text = input("Enter your symptoms: ")

# Preprocess the sample text
def preprocess_text(text):
    tokens = word_tokenize(text)
    snowball_stemmer = SnowballStemmer('english')
    tokens = [snowball_stemmer.stem(token.lower()) for token in tokens if token.isalpha()]
    return ' '.join(tokens)

sample_text_processed = preprocess_text(sample_text)

# Transform the preprocessed sample text using the loaded vectorizer
sample_text_transformed = tfidf_vectorizer.transform([sample_text_processed])

# Predict using the loaded model
predicted_label_encoded = voting_classifier.predict(sample_text_transformed)

# Decode the predicted label
predicted_label = label_encoder.inverse_transform(predicted_label_encoded)[0]
print("Predicted Label:", predicted_label)

# Define a dictionary mapping diseases to recommended tests and medications
disease_info = {
    "Psoriasis": {
        "tests": ["Skin biopsy", "Blood tests"],
        "medications": ["Neem", "Turmeric", "Aloe Vera", "Manjistha", "Guggulu"]
    },
    "Varicose Veins": {
        "tests": ["Doppler ultrasound", "Venography"],
        "medications": ["Horse chestnut", "Triphala", "Guggulu", "Brahmi", "Ashwagandha"]
    },
    "Typhoid": {
        "tests": ["Blood culture", "Widal test"],
        "medications": ["Giloy", "Neem", "Kutaj", "Triphala", "Chirayata"]
    },
    "Chickenpox": {
        "tests": ["Clinical diagnosis"],
        "medications": ["Neem", "Tulsi", "Amla", "Sandalwood", "Licorice"]
    },
    "Impetigo": {
        "tests": ["Clinical diagnosis"],
        "medications": ["Neem", "Manjistha", "Turmeric", "Triphala"]
    },
    "Dengue": {
        "tests": ["Dengue NS1 antigen test", "Dengue antibody test"],
        "medications": ["Papaya leaf extract", "Giloy", "Tulsi", "Turmeric", "Amla"]
    },
    "Fungal Infection": {
        "tests": ["Skin scraping", "KOH test"],
        "medications": ["Neem", "Manjistha", "Aloe Vera", "Turmeric", "Triphala"]
    },
    "Common Cold": {
        "tests": ["Clinical diagnosis"],
        "medications": ["Ginger", "Tulsi", "Cinnamon", "Turmeric", "Honey"]
    },
    "Pneumonia": {
        "tests": ["Chest X-ray", "Blood tests"],
        "medications": ["Tulsi", "Pippali", "Trikatu", "Licorice", "Guduch s i"]
    },
    "Arthritis": {
        "tests": ["X-ray", "Blood tests"],
        "medications": ["Ashwagandha", "Guggulu", "Shallaki", "Turmeric", "Ginger"]
    },
    "Acne": {
        "tests": ["Clinical diagnosis"],
        "medications": ["Neem", "Tulsi", "Aloe Vera", "Turmeric", "Manjistha"]
    },
    "Bronchial Asthma": {
        "tests": ["Spirometry", "Peak flow meter"],
        "medications": ["Vasaka", "Trikatu", "Tulsi", "Turmeric", "Ginger"]
    },
    "Hypertension": {
        "tests": ["Blood pressure measurement", "Blood tests"],
        "medications": ["Arjuna", "Ashwagandha", "Jatamansi", "Brahmi", "Gokshura"]
    },
    "Migraine": {
        "tests": ["Clinical diagnosis"],
        "medications": ["Lavender oil", "Peppermint oil", "Ginger", "Turmeric", "Feverfew"]
    },
    "Cervical Spondylosis": {
        "tests": ["X-ray", "MRI"],
        "medications": ["Ashwagandha", "Guggulu", "Bala", "Shallaki", "Ginger"]
    },
    "Jaundice": {
        "tests": ["Liver function tests", "Bilirubin test"],
        "medications": ["Kutki", "Bhumiamla", "Triphala", "Punarnava", "Guduchi"]
    },
    "Malaria": {
        "tests": ["Blood smear", "Rapid diagnostic test"],
        "medications": ["Giloy", "Neem", "Tulsi", "Chirayata", "Quinine"]
    },
    "Urinary Tract Infection": {
        "tests": ["Urine analysis", "Urine culture"],
        "medications": ["Punarnava", "Gokshura", "Neem", "Manjistha", "Triphala"]
    },
    "Allergy": {
        "tests": ["Skin prick test", "Blood tests"],
        "medications": ["Neem", "Turmeric", "Ginger", "Tulsi", "Guduchi"]
    },
    "Gastroesophageal Reflux Disease": {
        "tests": ["Endoscopy", "Esophageal manometry"],
        "medications": ["Aloe Vera", "Licorice", "Tulsi", "Ginger", "Peppermint oil"]
    },
    "Drug Reaction": {
        "tests": ["Clinical diagnosis"],
        "medications": ["Neem", "Tulsi", "Aloe Vera", "Manjistha", "Turmeric"]
    },
    "Peptic Ulcer Disease": {
        "tests": ["Endoscopy", "Urea breath test"],
        "medications": ["Licorice", "Aloe Vera", "Neem", "Tulsi", "Ginger"]
    },
    "Diabetes": {
        "tests": ["Fasting blood sugar test", "Oral glucose tolerance test"],
        "medications": ["Gudmar", "Fenugreek", "Turmeric", "Amla", "Neem"]
    },

}
def get_recommendations(predicted_label):
    if predicted_label in disease_info:
        return disease_info[predicted_label]["tests"], disease_info[predicted_label]["medications"]
    else:
        return [], []

recommended_tests, recommended_medications = get_recommendations(predicted_label)
print("Recommended Tests:", recommended_tests)
print("Recommended Medications:", recommended_medications)

# Function to predict disease from user input image
def predict_disease(image_path):
    # Preprocess the image
    img = preprocess_image(image_path)

    # Make predictions
    prediction = model.predict(img)

    # Get the predicted class index
    predicted_class_index = np.argmax(prediction)

    # Map the predicted class index to the corresponding class label
    class_labels = train_generator.class_indices
    predicted_disease = [key for key, value in class_labels.items() if value == predicted_class_index][0]

    return predicted_disease

# Get user input image path
user_image_path = input("Enter the path of the image: ")

# Predict disease for user input image
predicted_disease = predict_disease(user_image_path)

# Display prediction

print("Predicted disease:", predicted_disease)


In [ ]:
\

In [ ]:
import joblib
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import tkinter as tk
from tkinter import scrolledtext, messagebox
from PIL import ImageTk, Image
import numpy as np

# Load the disease prediction model and vectorizer
voting_classifier = joblib.load('voting_classifier_model_Disease_pred_97_percent_acc.pkl')
tfidf_vectorizer = joblib.load('tfidf_vectorizer_disease_nlp.joblib')
label_encoder = joblib.load('label_encoder_disease_nlp.joblib')

# Load the CNN model for image analysis (replace 'your_cnn_model.h5' with the path to your CNN model file)
# Assuming your CNN model is loaded using Keras
from keras.models import load_model
cnn_model = load_model('resnet.h5')

# Define a dictionary mapping diseases to recommended tests and medications
disease_info = {
    "Psoriasis": {
        "tests": ["Skin biopsy", "Blood tests"],
        "medications": ["Neem", "Turmeric", "Aloe Vera", "Manjistha", "Guggulu"]
    },
    "Varicose Veins": {
        "tests": ["Doppler ultrasound", "Venography"],
        "medications": ["Horse chestnut", "Triphala", "Guggulu", "Brahmi", "Ashwagandha"]
    },
    "Typhoid": {
        "tests": ["Blood culture", "Widal test"],
        "medications": ["Giloy", "Neem", "Kutaj", "Triphala", "Chirayata"]
    },
    "Chickenpox": {
        "tests": ["Clinical diagnosis"],
        "medications": ["Neem", "Tulsi", "Amla", "Sandalwood", "Licorice"]
    },
    "Impetigo": {
        "tests": ["Clinical diagnosis"],
        "medications": ["Neem", "Manjistha", "Turmeric", "Triphala"]
    },
    "Dengue": {
        "tests": ["Dengue NS1 antigen test", "Dengue antibody test"],
        "medications": ["Papaya leaf extract", "Giloy", "Tulsi", "Turmeric", "Amla"]
    },
    "Fungal Infection": {
        "tests": ["Skin scraping", "KOH test"],
        "medications": ["Neem", "Manjistha", "Aloe Vera", "Turmeric", "Triphala"]
    },
    "Common Cold": {
        "tests": ["Clinical diagnosis"],
        "medications": ["Ginger", "Tulsi", "Cinnamon", "Turmeric", "Honey"]
    },
    "Pneumonia": {
        "tests": ["Chest X-ray", "Blood tests"],
        "medications": ["Tulsi", "Pippali", "Trikatu", "Licorice", "Guduch s i"]
    },
    "Arthritis": {
        "tests": ["X-ray", "Blood tests"],
        "medications": ["Ashwagandha", "Guggulu", "Shallaki", "Turmeric", "Ginger"]
    },
    "Acne": {
        "tests": ["Clinical diagnosis"],
        "medications": ["Neem", "Tulsi", "Aloe Vera", "Turmeric", "Manjistha"]
    },
    "Bronchial Asthma": {
        "tests": ["Spirometry", "Peak flow meter"],
        "medications": ["Vasaka", "Trikatu", "Tulsi", "Turmeric", "Ginger"]
    },
    "Hypertension": {
        "tests": ["Blood pressure measurement", "Blood tests"],
        "medications": ["Arjuna", "Ashwagandha", "Jatamansi", "Brahmi", "Gokshura"]
    },
    "Migraine": {
        "tests": ["Clinical diagnosis"],
        "medications": ["Lavender oil", "Peppermint oil", "Ginger", "Turmeric", "Feverfew"]
    },
    "Cervical Spondylosis": {
        "tests": ["X-ray", "MRI"],
        "medications": ["Ashwagandha", "Guggulu", "Bala", "Shallaki", "Ginger"]
    },
    "Jaundice": {
        "tests": ["Liver function tests", "Bilirubin test"],
        "medications": ["Kutki", "Bhumiamla", "Triphala", "Punarnava", "Guduchi"]
    },
    "Malaria": {
        "tests": ["Blood smear", "Rapid diagnostic test"],
        "medications": ["Giloy", "Neem", "Tulsi", "Chirayata", "Quinine"]
    },
    "Urinary Tract Infection": {
        "tests": ["Urine analysis", "Urine culture"],
        "medications": ["Punarnava", "Gokshura", "Neem", "Manjistha", "Triphala"]
    },
    "Allergy": {
        "tests": ["Skin prick test", "Blood tests"],
        "medications": ["Neem", "Turmeric", "Ginger", "Tulsi", "Guduchi"]
    },
    "Gastroesophageal Reflux Disease": {
        "tests": ["Endoscopy", "Esophageal manometry"],
        "medications": ["Aloe Vera", "Licorice", "Tulsi", "Ginger", "Peppermint oil"]
    },
    "Drug Reaction": {
        "tests": ["Clinical diagnosis"],
        "medications": ["Neem", "Tulsi", "Aloe Vera", "Manjistha", "Turmeric"]
    },
    "Peptic Ulcer Disease": {
        "tests": ["Endoscopy", "Urea breath test"],
        "medications": ["Licorice", "Aloe Vera", "Neem", "Tulsi", "Ginger"]
    },
    "Diabetes": {
        "tests": ["Fasting blood sugar test", "Oral glucose tolerance test"],
        "medications": ["Gudmar", "Fenugreek", "Turmeric", "Amla", "Neem"]
    },
}
# Function to preprocess text
def preprocess_text(text):
    tokens = word_tokenize(text)
    snowball_stemmer = SnowballStemmer('english')
    tokens = [snowball_stemmer.stem(token.lower()) for token in tokens if token.isalpha()]
    return ' '.join(tokens)

# Function to preprocess image
def preprocess_image(image_path):
    # Add preprocessing steps as needed for your CNN model
    img = Image.open(image_path)
    # Resize the image to match the input size of your CNN model
    img = img.resize((224, 224))
    # Convert the image to array
    img_array = np.array(img)
    # Perform any other preprocessing steps required by your CNN model
    return img_array

# Function to predict disease and get recommendations
def predict_disease_and_recommendations(symptoms):
    # Preprocess the symptoms
    symptoms_processed = preprocess_text(symptoms)

    # Transform the preprocessed text using the loaded vectorizer
    symptoms_transformed = tfidf_vectorizer.transform([symptoms_processed])

    # Predict using the loaded model
    predicted_label_encoded = voting_classifier.predict(symptoms_transformed)

    # Decode the predicted label
    predicted_label = label_encoder.inverse_transform(predicted_label_encoded)[0]

    # Get recommended tests and medications for the predicted disease
    if predicted_label in disease_info:
        recommended_tests = ", ".join(disease_info[predicted_label]["tests"])
        recommended_medications = ", ".join(disease_info[predicted_label]["medications"])
    else:
        recommended_tests = "Not available"
        recommended_medications = "Not available"

    return predicted_label, recommended_tests, recommended_medications

# Function to process symptom prediction and CNN model analysis
def process_symptom_and_image(symptoms, image_path):
    # Predict disease and get recommendations
    predicted_disease, recommended_tests, recommended_medications = predict_disease_and_recommendations(symptoms)

    # Process image using the CNN model
    preprocessed_image = preprocess_image(image_path)
    cnn_predictions = cnn_model.predict(np.expand_dims(preprocessed_image, axis=0))

    return predicted_disease, recommended_tests, recommended_medications, cnn_predictions

# Function to handle the 'Predict' button click
def predict_button_click():
    symptoms = symptoms_entry.get("1.0", tk.END).strip()
    if symptoms:
        # If CNN model path is provided and predicted disease is in the list, ask for image and process
        if cnn_model_path:
            image_path = ask_for_image()
            if image_path:
                predicted_disease, recommended_tests, recommended_medications, cnn_predictions = process_symptom_and_image(symptoms, image_path)
                messagebox.showinfo("Prediction Result", f"Predicted Disease: {predicted_disease}\n\nRecommended Tests: {recommended_tests}\n\nRecommended Medications: {recommended_medications}")
                # Show the output of the CNN model
                print("CNN Predictions:", cnn_predictions)
        else:
            predicted_disease, recommended_tests, recommended_medications = predict_disease_and_recommendations(symptoms)
            messagebox.showinfo("Prediction Result", f"Predicted Disease: {predicted_disease}\n\nRecommended Tests: {recommended_tests}\n\nRecommended Medications: {recommended_medications}")
            # Store patient data in the GUI
            patient_data_text.insert(tk.END, f"\n\nSymptoms: {symptoms}\nPredicted Disease: {predicted_disease}\nRecommended Tests: {recommended_tests}\nRecommended Medications: {recommended_medications}")
    else:
        messagebox.showerror("Error", "Please enter your symptoms.")

# Create the main window
root = tk.Tk()
root.title("Disease Prediction Chatbot")

# Create text area for entering symptoms
symptoms_label = tk.Label(root, text="Enter your symptoms:")
symptoms_label.pack()
symptoms_entry = scrolledtext.ScrolledText(root, width=40, height=5, wrap=tk.WORD)
symptoms_entry.pack()

# Create Predict button
predict_button = tk.Button(root, text="Predict", command=predict_button_click)
predict_button.pack()

# Create text area to display patient data
patient_data_label = tk.Label(root, text="Patient Data:")
patient_data_label.pack()
patient_data_text = scrolledtext.ScrolledText(root, width=60, height=15, wrap=tk.WORD)
patient_data_text.pack()

# Run the main event loop
root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/80/jd0146wj1sn0f47f5phvs8t00000gn/T/ipykernel_9162/3133541118.py", line 172, in predict_button_click
    if cnn_model_path:
       ^^^^^^^^^^^^^^
NameError: name 'cnn_model_path' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/80/jd0146wj1sn0f47f5phvs8t00000gn/T/ipykernel_9162/3133541118.py", line 172, in predict_button_click
    if cnn_model_path:
       ^^^^^^^^^^^^^^
NameError: name 'cnn_model_path' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return sel

KeyboardInterrupt: 

In [ ]:
import tkinter as tk
from tkinter import scrolledtext, messagebox, filedialog
import joblib
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from keras.models import load_model
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load NLP model and other necessary resources
voting_classifier = joblib.load('voting_classifier_model_Disease_pred_97_percent_acc.pkl')
tfidf_vectorizer = joblib.load('tfidf_vectorizer_disease_nlp.joblib')
label_encoder = joblib.load('label_encoder_disease_nlp.joblib')

# Load CNN model
cnn_model = load_model('resnet.h5')  # Replace 'resnet.h5' with the path to your CNN model file

# Define a dictionary mapping diseases to recommended tests and medications
# (You can use the same disease_info dictionary from before)

# Function to preprocess text for NLP model
def preprocess_text(text):
    tokens = word_tokenize(text)
    snowball_stemmer = SnowballStemmer('english')
    tokens = [snowball_stemmer.stem(token.lower()) for token in tokens if token.isalpha()]
    return ' '.join(tokens)

# Function to predict disease and get recommendations using NLP model
def predict_disease_and_recommendations(symptoms):
    # Preprocess the symptoms
    symptoms_processed = preprocess_text(symptoms)

    # Transform the preprocessed text using the loaded vectorizer
    symptoms_transformed = tfidf_vectorizer.transform([symptoms_processed])

    # Predict using the loaded NLP model
    predicted_label_encoded = voting_classifier.predict(symptoms_transformed)

    # Decode the predicted label
    predicted_label = label_encoder.inverse_transform(predicted_label_encoded)[0]

    # Get recommended tests and medications for the predicted disease
    if predicted_label in disease_info:
        recommended_tests = ", ".join(disease_info[predicted_label]["tests"])
        recommended_medications = ", ".join(disease_info[predicted_label]["medications"])
    else:
        recommended_tests = "Not available"
        recommended_medications = "Not available"

    return predicted_label, recommended_tests, recommended_medications

# Function to handle the 'Predict' button click for NLP model
def predict_button_click_nlp():
    symptoms = symptoms_entry.get("1.0", tk.END).strip()
    if symptoms:
        predicted_disease, recommended_tests, recommended_medications = predict_disease_and_recommendations(symptoms)
        messagebox.showinfo("Prediction Result (NLP Model)", f"Predicted Disease: {predicted_disease}\n\nRecommended Tests: {recommended_tests}\n\nRecommended Medications: {recommended_medications}")
        # Store patient data in the GUI
        patient_data_text.insert(tk.END, f"\n\nSymptoms: {symptoms}\nPredicted Disease: {predicted_disease}\nRecommended Tests: {recommended_tests}\nRecommended Medications: {recommended_medications}")
    else:
        messagebox.showerror("Error", "Please enter your symptoms.")

# Function to handle the 'Predict' button click for CNN model
def predict_button_click_cnn():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg")])
    if file_path:
        img = cv2.imread(file_path)
        img = cv2.resize(img, (224, 224))
        img = img / 255.0  # Normalize pixel values
        img = np.expand_dims(img, axis=0)  # Add batch dimension
        prediction = cnn_model.predict(img)
        predicted_class_index = np.argmax(prediction)
        predicted_class_name = class_names[predicted_class_index]
        plt.imshow(cv2.cvtColor(cv2.imread(file_path), cv2.COLOR_BGR2RGB))
        plt.title(f"Predicted Disease: {predicted_class_name}")
        plt.axis('off')
        plt.show()

# Create the main window
root = tk.Tk()
root.title("Disease Prediction")

# Create text area for entering symptoms
symptoms_label = tk.Label(root, text="Enter your symptoms:")
symptoms_label.pack()
symptoms_entry = scrolledtext.ScrolledText(root, width=40, height=5, wrap=tk.WORD)
symptoms_entry.pack()

# Create Predict button for NLP model
predict_button_nlp = tk.Button(root, text="Predict (NLP Model)", command=predict_button_click_nlp)
predict_button_nlp.pack()

# Create Predict button for CNN model
predict_button_cnn = tk.Button(root, text="Predict (CNN Model)", command=predict_button_click_cnn)
predict_button_cnn.pack()

# Create text area to display patient data
patient_data_label = tk.Label(root, text="Patient Data:")
patient_data_label.pack()
patient_data_text = scrolledtext.ScrolledText(root, width=60, height=15, wrap=tk.WORD)
patient_data_text.pack()

# Run the main event loop
root.mainloop()


2024-05-02 11:51:42.044038: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/80/jd0146wj1sn0f47f5phvs8t00000gn/T/ipykernel_9162/2960075484.py", line 57, in predict_button_click_nlp
    predicted_disease, recommended_tests, recommended_medications = predict_disease_and_recommendations(symptoms)
                                                                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/80/jd0146wj1sn0f47f5phvs8t00000gn/T/ipykernel_9162/2960075484.py", line 44, in predict_disease_and_recommendat

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/80/jd0146wj1sn0f47f5phvs8t00000gn/T/ipykernel_9162/2960075484.py", line 74, in predict_button_click_cnn
    predicted_class_name = class_names[predicted_class_index]
                           ^^^^^^^^^^^
NameError: name 'class_names' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/80/jd0146wj1sn0f47f5phvs8t00000gn/T/ipykernel_9162/2960075484.py", line 57, in predict_button_click_nlp
    predicted_disease, recommended_tests, recommended_medications = predict_disease_and_recommendations(symptoms)
                                                                    ^^^^^^^^^^^^^^